## (Positive) Point Mutual Information

Point Mutual Information (PMI) 는 (word, contexts) 이나 (input, outputs) 와의 상관성을 측정하는 방법입니다. 두 변수 $x$, $y$ 의 상관성은 다음처럼 정의됩니다. 서로 상관이 없는 변수 $x$, $y$의 pmi 는 0 이며, 그 값이 클수록 positive correlated 있습니다. 

$pmi(x,y) = log \left( \frac{p(x,y)}{p(x) \times p(y)} \right )$

Positive PMI 는 음의 값을 지니는 PMI 를 모두 0으로 치환합니다. 

$ppmi(x,y) = max(0, log \left( \frac{p(x,y)}{p(x) \times p(y)} \right)$

그런데 PMI 는 infrequent $y$ 에 대하여 그 값이 지나치게 예민합니다. 이를 보완하기 위하여 smoothing 을 할 수 있습니다. soynlp 에서는 다음과 같은 smoothing 방법을 이용합니다. $\alpha$ 를 $p(y)$ 에 더합니다. 

$pmi(x,y) = log \left( \frac{p(x,y)}{p(x) \times \left( p(y) + \alpha \right)} \right)$

$\alpha$ 는 y 의 threshold 역할을 합니다. PMI 는 다음처럼 기술될 수 있습니다. $p(y)$ 가 $\alpha$ 보다 큰 값들이 positive pmi value 를 지닐 수 있습니다. 

$pmi(x,y) = \frac{p(y \vert x)}{\left( p(y) + \alpha \right)}$

PPMI 를 위해서 min_pmi 를 기준으로 threshold cutting 을 하는 기능도 제공합니다. 

## Word - context matrix

Word - context 그래프는 단어의 문맥을 파악하기 위해 사용될 수 있습니다. 

sentence = ['a', 'little', 'cat', 'sit', 'on', 'table'] , window = 2 일 때, 'cat' 의 context words 는 ['a', 'little', 'sit', 'on'] 입니다. sent_to_word_context_matrix() 함수는 이 역할을 수행합니다. min_tf 는 minimum frequency 입니다. 

Return 은 scipy.sparse.csr.csr_matrix 형식의 word - context matrix 와 list of str 형식의 vocabulary list 입니다. 

In [1]:
import sys
sys.path.append('../')

import soynlp
print(soynlp.__version__)

0.0.46


사용할 토크나이저를 학습합니다. 

In [2]:
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

corpus_path = '/mnt/lovit/works/fastcampus_text_ml/2nd/data/corpus_10days/news/2016-10-20_article_all_normed.txt'
corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)
print('num sents = {}'.format(len(corpus)))

word_extractor = WordExtractor()
word_extractor.train(corpus)
cohesions = word_extractor.all_cohesion_scores()
print(cohesions['뉴스'])

l_cohesions = {word:score[0] for word, score in cohesions.items()}
tokenizer = LTokenizer(l_cohesions)
print(tokenizer('하루의 뉴스를 학습했습니다'))

num sents = 223357
training was done. used memory 0.717 Gbse memory 0.770 Gb
all cohesion probabilities was computed. # words = 223348
(0.487322733132789, 0.22771099423991986)
['하루', '의', '뉴스', '를', '학습', '했습니다']


sent_to_word_contexts_matrix() 에 window, min_tf, tokenizer 를 넣습니다. verbose=True 이면 vectorizing 되는 상태의 모니터링이 가능합니다. 

In [3]:
from soynlp.vectorizer import sent_to_word_contexts_matrix

x, idx2vocab = sent_to_word_contexts_matrix(
    corpus,
    windows=3,
    min_tf=10,
    tokenizer=tokenizer, # (default) lambda x:x.split(),
    dynamic_weight=False,
    verbose=True)

Create (word, contexts) matrix
  - counting word frequency from 223356 sents, mem=0.720 Gb
  - scanning (word, context) pairs from 223356 sents, mem=1.177 Gb
  - (word, context) matrix was constructed. shape = (48583, 48583)                    
  - done


## PMI

soynlp.word.pmi 는 x 의 (rows, columns) 에 대한 pmi 를 계산합니다. row 가 x, column 이 y 입니다. 

In [4]:
from soynlp.word import pmi

pmi_dok = pmi(
    x,
    min_pmi=0,
    alpha=0.0001,
    verbose=True)

computing pmi was done   


단어 '이화여대'와 pmi 가 높은 (트와이스 주변에 자주 등장한) 단어를 찾습니다. 

In [5]:
vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
query = vocab2idx['이화여대']

submatrix = pmi_dok[query,:].tocsr() # get the row of query
contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
pmi_i = submatrix.data

most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:10]
most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds]

from pprint import pprint
pprint(most_relateds)

[('최경희', 5.264477011557446),
 ('총장이', 5.1888921657268945),
 ('서대문구', 4.620777810284549),
 ('특혜', 4.357461105741046),
 ('딸', 4.227805130145214),
 ('모모영화관에서', 4.141379012627799),
 ('아트하우스', 4.106686034264577),
 ('입학', 4.07980530118817),
 ('정유라', 4.070438044177448),
 ('교수', 4.024990303114813)]


단어 '이화여대'와 유사한 contexts vector 를 지닌 단어를 찾습니다. 

In [6]:
from sklearn.metrics import pairwise_distances

dist = pairwise_distances(x[query, :], x, metric='cosine')
most_similars = dist.argsort()[:10] # sorting and return index
print(most_similars)

most_similars = most_similars[0][:10] # only a row
print(most_similars)

[[ 1129  2614 36331 ...   633 27658 32682]]
[ 1129  2614 36331 19630   337 37516   884   374   888 11558]


context 가 비슷한 다른 단어를 idx 에서 str 로 변환합니다. pairwise_distances 는 cosine distance 를 이용하였기 때문에, cosine similarity 로 변환하기 위하여 1 - distance 를 합니다. 

In [7]:
for similar_idx in most_similars:
    d = dist[0, similar_idx]
    similar_word = idx2vocab[similar_idx]
    similarity = 1 - d
    print('{} = {}'.format(similar_word, similarity))

이화여대 = 0.9999999999999903
이대 = 0.7298236227304
사퇴했지만 = 0.548733330672858
사퇴한 = 0.5208347062837736
교수 = 0.5102588848338734
사임했습니다 = 0.509711793795689
총장 = 0.5067719153392938
학교 = 0.5040449859535396
최 = 0.4995619813125106
정씨 = 0.4920096515861163


## PMI module

most similar words 나 most related context 를 쉽게 확인하기 위하여, 위의 두 기능을 포함한 class 를 제공합니다. 

In [8]:
from soynlp.word import PMI

pmi_trainer = PMI(
    windows=3,
    min_tf=10,
    verbose=True,
    tokenizer=tokenizer, # (default) lambda x:x.split()
    min_pmi=0,
    alpha=0.0001
)

pmi_trainer.train(corpus)

(word, context) matrix was constructed. shape = (48583, 48583)                    
computing pmi was done   


soynlp.word.PMI.x 에는 word - context matrix 가 저장되어 있습니다. scipy.sparse.csr.csr_matrix 형식입니다. 

soynlp.word.PMI.idx2vocab 는 word - context 의 각 차원에 해당하는 단어를 포함한 list of str 입니다. 

soynlp.word.PMI.vocab2idx 는 word - context 에 포함된 단어의 index 가 저장된 dict 입니다. {word:idx} 의 정보가 저장되어 있습니다. 

soynlp.word.PMI.pmi\_ 에는 word, context 간의 pmi value 가 저장되어 있습니다. scipy.sparse.dok.dok_matrix 형식입니다. 

In [9]:
print(pmi_trainer.x.shape)
print(len(pmi_trainer.idx2vocab))
print(len(pmi_trainer.vocab2idx))
print(pmi_trainer.pmi_.shape)

(48583, 48583)
48583
48583
(48583, 48583)


most_similar_words(query, topk=10) 는 cosine distance 기준, query 와 context 가 비슷한 topk 개의 단어를 찾습니다. 

In [10]:
pprint(pmi_trainer.most_similar_words('이화여대'))

[('이대', 0.7298236227304),
 ('사퇴했지만', 0.548733330672858),
 ('사퇴한', 0.5208347062837736),
 ('교수', 0.5102588848338734),
 ('사임했습니다', 0.509711793795689),
 ('총장', 0.5067719153392938),
 ('학교', 0.5040449859535396),
 ('최', 0.4995619813125106),
 ('정씨', 0.4920096515861163),
 ('정유라씨', 0.4879237903414544)]


most_related_contexts(query, topk=10) 는 query 와 pmi 가 가장 큰 topk 개의 contexts 를 찾습니다. 

In [11]:
pprint(pmi_trainer.most_related_contexts('이화여대'))

[('최경희', 5.264477011557446),
 ('총장이', 5.1888921657268945),
 ('서대문구', 4.620777810284549),
 ('특혜', 4.357461105741046),
 ('딸', 4.227805130145214),
 ('모모영화관에서', 4.141379012627799),
 ('아트하우스', 4.106686034264577),
 ('입학', 4.07980530118817),
 ('정유라', 4.070438044177448),
 ('교수', 4.024990303114813)]
